# Step 7 - Deploy you model

In step 5 we created CI pipeline with jobs `build`, `test` and `experiments`.

Here we'll add new job `deploy` for deploying model on commit to branch `experiments`

# Add `deploy` job to CI/CD pipeline 


## Add new stage

```yaml

stages:
  - build
  - test
  - experiments
  - deploy # <- add new stage 
```


### Add deploy job

```yaml

deploy:
  tags:
    - shell
  only:
    - experiments
  stage: deploy
  script:
    - cd /tmp/$CI_PROJECT_NAME
    - git fetch
    - git checkout $CI_BUILD_REF_NAME --
    - git pull origin $CI_BUILD_REF_NAME
    - dvc fetch
    - dvc checkout
    - |
      if ! curl http://localhost:9000/healthcheck; then
        docker run \
            -d \
            --name=deploy-sklearn-$(date +"%Y-%m-%d-%H-%M-%S") \
            -v $(pwd)/$PROJECT_MODEL_PATH:$DOCKER_MODEL_PATH \
            -e MODEL_PATH=$DOCKER_MODEL_PATH \
            -p 9000:9000 \
                mlrepa/deploy-sklearn:latest
      else
        curl -X POST http://localhost:9000/reload-model
      fi
```

## Full `.gitlab-ci.yml`

```yaml
variables:
  DEPLOY_IMAGE: mlrepa/deploy-sklearn:latest
  PROJECT_MODEL_PATH: models/model.joblib
  DOCKER_MODEL_PATH: /home/deploy/models/model.joblib

stages:
  - build
  - test
  - experiments
  - deploy

build:
  stage: build
  tags:
    - shell
  only:
    - master
    - test
  script:
    - docker pull $DEPLOY_IMAGE
    - |
      if [ ! -d "/tmp/$CI_PROJECT_NAME" ]; then
        project_url=${CI_PROJECT_URL//"https://"/}
        git clone \
          https://oauth2:$repo_token@${project_url}.git \
          "/tmp/$CI_PROJECT_NAME"
      fi
      cd /tmp/$CI_PROJECT_NAME
      git checkout $CI_BUILD_REF_NAME
      git pull origin $CI_BUILD_REF_NAME
      dvc fetch
      dvc checkout

test:
  stage: test
  tags:
    - shell
  only:
    - master
    - test
  script:
    - cd /tmp/$CI_PROJECT_NAME
    - |
      docker run \
            --name=deploy-sklearn-test \
            --rm \
            -v $(pwd)/tests:/home/deploy/tests \
            -v $(pwd)/$PROJECT_MODEL_PATH:$DOCKER_MODEL_PATH \
            -e MODEL_PATH=$DOCKER_MODEL_PATH \
                $DEPLOY_IMAGE /bin/bash -c "
                    python /home/deploy/tests/test_model.py
                "

cml:
    stage: experiments
    image: mlrepa/cml:latest #dvcorg/cml-py3:latest
    only:
      - experiments
    tags:
      - docker,experiments
    script:
      - apt-get install -y jq
      - git fetch
      - git checkout $CI_BUILD_REF_NAME --
      - git pull origin $CI_BUILD_REF_NAME
      - dvc fetch
      - dvc checkout
      - dvc repro
      - |
        echo "## f1_score: $(cat experiments/metrics.json | jq '.f1_score')" >> report.md
      - dvc metrics diff --show-md >> report.md
      - echo >> report.md
      - cml-publish experiments/confusion_matrix.png --md >> report.md
      - cml-send-comment report.md

deploy:
  tags:
    - shell
  only:
    - experiments
  stage: deploy
  script:
    - cd /tmp/$CI_PROJECT_NAME
    - git fetch
    - git checkout $CI_BUILD_REF_NAME --
    - git pull origin $CI_BUILD_REF_NAME
    - dvc fetch
    - dvc checkout
    - |
      if ! curl http://localhost:9000/healthcheck; then
        docker run \
            -d \
            --name=deploy-sklearn-$(date +"%Y-%m-%d-%H-%M-%S") \
            -v $(pwd)/$PROJECT_MODEL_PATH:$DOCKER_MODEL_PATH \
            -e MODEL_PATH=$DOCKER_MODEL_PATH \
            -p 9000:9000 \
                mlrepa/deploy-sklearn:latest
      else
        curl -X POST http://localhost:9000/reload-model
      fi
```

# Push commits to GitLab repo

```bash
git add .gitlab-ci.yml
git commit -m "Add CI job 'deploy'"
git push origin experiments
```

# Test deployment (on local HOST) 

## Check connection


```bash 

curl http://localhost:9000/healthcheck
 
```

## Get predictions


```bash 

curl http://localhost:9000/predict \
    -d '{"data":[[value11,value12,...,value1N],...,[value1M,valueM2,...,valueMN]]}'

``` 
        
where:

M - number if rows to predict;
N - number if columns.

Columns order is the same as in train dataset.

Example:

```bash

curl -POST http://localhost:9000/predict \
     -d '{"data":[[4,3,2,1,0,0,1,2,3,4,2,2], [1,3,2,1,2,0,1,2,3,4,2,8]]}'
        
```